# Making predictions using the pre-trained ImageNet classifier

There are two parts of an imagine classification network:

The neural network begins witn a series of pooling and convolution layers, then ends with a densely connect classifier. The models in the `keras.applications` package ship with dense layer specialized towards performance against the ImageNet dataset.

In [ ]:
from keras.applications.mobilenetv2 import MobileNetV2


conv_base = MobileNetV2(
    weights='imagenet',
    include_top=True # include the densely connected classifer, which sits on top of hte convolutional network
)

conv_base.summary()

In [ ]:
# from ipywebrtc import VideoStream, ImageRecorder
from io import BytesIO
from time import sleep
from picamera import PiCamera
import time
import numpy as np
from datetime import datetime
from picamera.array import PiRGBArray

from keras.applications.mobilenetv2 import preprocess_input, decode_predictions

print('***')


def extract_features(frame, frame_count=1, classifier_count=1000):
    # expand 3D RGB frame into 4D "batch"
    sample = np.expand_dims(frame, axis=0)
    processed_sample = preprocess_input(sample.astype(np.float32))
    print('expanded to 4D array')
    features = conv_base.predict(processed_sample)
    print('extracted features')
    decoded_features = decode_predictions(features)
    print('reshaped features')
    print(decoded_features)
    return decoded_features

    
                                   
with PiCamera() as camera:
    camera.resolution = (320, 240)
    camera.framerate = 24
    time.sleep(2)
    # output = np.empty((224, 224, 3), dtype=np.float32)
    print('initialized output')
    # camera.capture(output, 'rgb')
    ts = datetime.now()
    camera.capture('data/{0}.jpg'.format((ts)))

    
    stream = PiRGBArray(camera, size=(320, 240))
    camera.capture(stream, format='bgr')
        # At this point the image is available as stream.array
    frame = stream.array
    frame = frame[0:224, 48:272, :]  # crop the frame

    features = extract_features(frame)

    

    